In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

C:\Users\mjaye\miniconda3\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\mjaye\miniconda3\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\mjaye\miniconda3\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the gencode to avoid compatibility violations i

In [2]:
df = pd.read_csv('uncleaned_final_final.csv')
df

,Unnamed: 0,Normalized EBITDA,Reconciled Depreciation,EBITDA,EBIT,Interest Expense,Diluted Average Shares,Diluted EPS,Net Income,Tax Provision,Other Non Operating Income Expenses,Operating Income,Operating Expense,Other Operating Expenses,Gross Profit,Cost Of Revenue,Total Revenue,Operating Revenue,Ticker,Sector
0,2025-06-30,2.160000e+09,1.140000e+09,2.160000e+09,1.020000e+09,600000000.0,3.613445e+08,1.19,4.300000e+08,-2.000000e+07,4.000000e+07,8.100000e+08,4.720000e+09,2.490000e+09,5.530000e+09,1.122000e+10,1.675000e+10,1.675000e+10,AARTIIND.NS,Chemicals
1,2025-03-31,2.374600e+09,1.132700e+09,2.268000e+09,1.135300e+09,253000000.0,NaN,NaN,9.587000e+08,-7.630000e+07,-3.318000e+08,1.868400e+09,-3.518700e+09,3.300000e+09,-1.650300e+09,1.973350e+10,1.808320e+10,1.808320e+10,AARTIIND.NS,Chemicals
2,2024-12-31,2.360000e+09,1.110000e+09,2.360000e+09,1.250000e+09,850000000.0,3.622047e+08,1.27,4.600000e+08,-6.000000e+07,5.000000e+07,1.210000e+09,4.890000e+09,2.730000e+09,6.100000e+09,1.230000e+10,1.840000e+10,1.840000e+10,AARTIIND.NS,Chemicals
3,2024-09-30,2.020000e+09,1.080000e+09,2.040000e+09,9.600000e+08,620000000.0,3.611111e+08,1.44,5.200000e+08,-1.800000e+08,5.000000e+07,8.800000e+08,5.230000e+09,3.100000e+09,6.110000e+09,1.017000e+10,1.628000e+10,1.628000e+10,AARTIIND.NS,Chemicals
4,2024-06-30,3.110000e+09,1.020000e+09,3.110000e+09,2.090000e+09,640000000.0,3.633952e+08,3.77,1.370000e+09,8.000000e+07,6.000000e+07,1.980000e+09,4.960000e+09,2.740000e+09,6.940000e+09,1.157000e+10,1.851000e+10,1.851000e+10,AARTIIND.NS,Chemicals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,2025-06-30,2.243400e+10,2.381000e+09,2.243400e+10,2.005300e+10,847000000.0,1.006036e+09,14.58,1.466800e+10,4.340000e+09,1.549000e+09,1.793300e+10,2.884200e+10,1.747600e+10,4.677500e+10,1.789500e+10,6.467000e+10,6.467000e+10,ZYDUSLIFE.NS,Healthcare
1504,2025-03-31,1.990900e+10,2.379000e+09,1.914100e+10,1.676200e+10,42000000.0,NaN,NaN,1.170900e+10,4.232000e+09,-1.800000e+09,1.927000e+10,2.337700e+10,-1.879200e+10,4.264700e+10,2.025500e+10,6.290200e+10,6.290200e+10,ZYDUSLIFE.NS,Healthcare
1505,2024-12-31,1.445100e+10,2.290000e+09,1.445100e+10,1.216100e+10,320000000.0,1.006391e+09,10.17,1.023500e+10,1.795000e+09,5.750000e+08,9.760000e+09,2.564100e+10,1.534200e+10,3.540100e+10,1.583400e+10,5.123500e+10,5.123500e+10,ZYDUSLIFE.NS,Healthcare
1506,2024-09-30,1.529600e+10,2.336000e+09,1.529600e+10,1.296000e+10,251000000.0,1.005740e+09,9.06,9.112000e+09,3.731000e+09,6.820000e+08,1.182400e+10,2.459800e+10,1.470400e+10,3.642200e+10,1.471400e+10,5.113600e+10,5.113600e+10,ZYDUSLIFE.NS,Healthcare


In [3]:
important_cols = [
    "Total Revenue",
    "Operating Revenue",
    "Gross Profit",
    "EBITDA",
    "EBIT",
    "Operating Income",
    "Net Income",
    "Operating Expense",
    "Cost Of Revenue",
    "Diluted EPS"
]

def corrupted_or_missing(x):
    """
    Returns True if column for this ticker is completely unusable:
    - All values NaN
    - All values 0 (or a single repeated constant)
    """
    return x.isna().all() or (x.nunique(dropna=True) == 1 and (x.iloc[0] in [0, -9999]))

# Apply check ticker-wise
mask = df.groupby("Ticker").apply(
    lambda g: g.apply(corrupted_or_missing)
).any(axis=1)

# Tickers to drop
tickers_to_drop = mask[mask].index.tolist()

print(len(tickers_to_drop))

# Cleaned dataframe
df = df[~df["Ticker"].isin(tickers_to_drop)]

63


C:\Users\mjaye\AppData\Local\Temp\ipykernel_9928\3039813572.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mask = df.groupby("Ticker").apply(


In [4]:
df = df[df['Sector'].notna()]

In [6]:
def impute_financials(df, group_key="Ticker"):
    df_out = df.copy()
    
    # interpolate within each ticker
    df_out = df_out.groupby(group_key).apply(
        lambda g: g.interpolate(method="linear", limit_direction="both")
    ).reset_index(drop=True)
    
    # fill remaining NaN with sector/overall median
    df_out = df_out.fillna(df_out.median(numeric_only=True))
    
    return df_out

In [7]:
df = impute_financials(df)

C:\Users\mjaye\AppData\Local\Temp\ipykernel_9928\527018751.py:6: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  lambda g: g.interpolate(method="linear", limit_direction="both")
C:\Users\mjaye\AppData\Local\Temp\ipykernel_9928\527018751.py:6: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  lambda g: g.interpolate(method="linear", limit_direction="both")
C:\Users\mjaye\AppData\Local\Temp\ipykernel_9928\527018751.py:6: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  lambda g: g.interpolate(method="linear", limit_direction="both")
C:\Users\mjaye\AppData\Local\Temp\ipykernel_9928\527018751.py:6: FutureWarning: DataFrame.inte

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1194 entries, 0 to 1193
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Unnamed: 0                           1194 non-null   object 
 1   Normalized EBITDA                    1194 non-null   float64
 2   Reconciled Depreciation              1194 non-null   float64
 3   EBITDA                               1194 non-null   float64
 4   EBIT                                 1194 non-null   float64
 5   Interest Expense                     1194 non-null   float64
 6   Diluted Average Shares               1194 non-null   float64
 7   Diluted EPS                          1194 non-null   float64
 8   Net Income                           1194 non-null   float64
 9   Tax Provision                        1194 non-null   float64
 10  Other Non Operating Income Expenses  1194 non-null   float64
 11  Operating Income              

In [9]:
df["Unnamed: 0"] = pd.to_datetime(df["Unnamed: 0"], errors="coerce")
df.rename(columns={"Unnamed: 0": "Date"}, inplace=True)

In [10]:
df = df.sort_values(by=["Ticker", "Date"]).reset_index(drop=True)

In [11]:
# Total unique quarters in dataset
print("Unique quarters in dataset:", df["Date"].nunique())

# Count how many unique quarters per ticker
quarter_counts = df.groupby("Ticker")["Date"].nunique()

# Summary stats
print(quarter_counts.describe())

# Tickers with < 5 quarters (incomplete data)
missing_quarters = quarter_counts[quarter_counts < 5]
print("Tickers with missing quarters:\n", missing_quarters)

# Split into mildly vs heavily corrupted
missing_one = quarter_counts[quarter_counts == 4]
heavily_corrupted = quarter_counts[quarter_counts <= 3]

print("\nTickers missing exactly 1 quarter:\n", missing_one)
print("\nHeavily corrupted tickers:\n", heavily_corrupted)

Unique quarters in dataset: 5
count    239.000000
mean       4.995816
std        0.064685
min        4.000000
25%        5.000000
50%        5.000000
75%        5.000000
max        5.000000
Name: Date, dtype: float64
Tickers with missing quarters:
 Ticker
PGHH.NS    4
Name: Date, dtype: int64

Tickers missing exactly 1 quarter:
 Ticker
PGHH.NS    4
Name: Date, dtype: int64

Heavily corrupted tickers:
 Series([], Name: Date, dtype: int64)


In [12]:
df = df[df["Ticker"] != "PGHH.NS"]

In [13]:
df['Gross_Profit_Margin'] = (df['Gross Profit'] / df['Total Revenue'].replace(0, np.nan)).fillna(0)
df['Operating_Margin'] = (df['Operating Income'] / df['Total Revenue'].replace(0, np.nan)).fillna(0)
df['Net_Profit_Margin'] = (df['Net Income'] / df['Total Revenue'].replace(0, np.nan)).fillna(0)
df['EBITDA_Margin'] = (df['Normalized EBITDA'] / df['Total Revenue'].replace(0, np.nan)).fillna(0)

df['Expense_Ratio'] = (df['Operating Expense'] / df['Total Revenue'].replace(0, np.nan)).fillna(0)

pretax_income = df['Net Income'] + df['Tax Provision']
df['Effective_Tax_Rate'] = (df['Tax Provision'] / pretax_income.replace(0, np.nan)).fillna(0)

df = df.sort_values(by=['Ticker', 'Date']) # Make sure data is sorted correctly
df['Revenue_Growth_QoQ'] = df.groupby('Ticker')['Total Revenue'].pct_change().fillna(0)
df['Net_Income_Growth_QoQ'] = df.groupby('Ticker')['Net Income'].pct_change().fillna(0)
df['Gross_Profit_Growth_QoQ'] = df.groupby('Ticker')['Gross Profit'].pct_change().fillna(0)

df.replace([np.inf, -np.inf], 0, inplace=True)
df.to_csv('final_engineered.csv', index=False)

Starting feature engineering...


In [14]:
df = pd.read_csv('final_engineered.csv')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1190 entries, 0 to 1189
Data columns (total 29 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Date                                 1190 non-null   object 
 1   Normalized EBITDA                    1190 non-null   float64
 2   Reconciled Depreciation              1190 non-null   float64
 3   EBITDA                               1190 non-null   float64
 4   EBIT                                 1190 non-null   float64
 5   Interest Expense                     1190 non-null   float64
 6   Diluted Average Shares               1190 non-null   float64
 7   Diluted EPS                          1190 non-null   float64
 8   Net Income                           1190 non-null   float64
 9   Tax Provision                        1190 non-null   float64
 10  Other Non Operating Income Expenses  1190 non-null   float64
 11  Operating Income              

In [16]:
df.head()

,Date,Normalized EBITDA,Reconciled Depreciation,EBITDA,EBIT,Interest Expense,Diluted Average Shares,Diluted EPS,Net Income,Tax Provision,...,Sector,Gross_Profit_Margin,Operating_Margin,Net_Profit_Margin,EBITDA_Margin,Expense_Ratio,Effective_Tax_Rate,Revenue_Growth_QoQ,Net_Income_Growth_QoQ,Gross_Profit_Growth_QoQ
0,2024-06-30,3.110000e+09,1.020000e+09,3.110000e+09,2.090000e+09,640000000.0,363395225.0,3.77,1.370000e+09,80000000.0,...,Chemicals,0.374932,0.106969,0.074014,0.168017,0.267963,0.055172,0.000000,0.000000,0.000000
1,2024-09-30,2.020000e+09,1.080000e+09,2.040000e+09,9.600000e+08,620000000.0,361111111.0,1.44,5.200000e+08,-180000000.0,...,Chemicals,0.375307,0.054054,0.031941,0.124079,0.321253,-0.529412,-0.120475,-0.620438,-0.119597
2,2024-12-31,2.360000e+09,1.110000e+09,2.360000e+09,1.250000e+09,850000000.0,362204724.0,1.27,4.600000e+08,-60000000.0,...,Chemicals,0.331522,0.065761,0.025000,0.128261,0.265761,-0.150000,0.130221,-0.115385,-0.001637
3,2025-03-31,2.374600e+09,1.132700e+09,2.268000e+09,1.135300e+09,253000000.0,361774631.0,1.23,9.587000e+08,-76300000.0,...,Chemicals,-0.091262,0.103322,0.053016,0.131315,-0.194584,-0.086469,-0.017217,1.084130,-1.270541
4,2025-06-30,2.160000e+09,1.140000e+09,2.160000e+09,1.020000e+09,600000000.0,361344538.0,1.19,4.300000e+08,-20000000.0,...,Chemicals,0.330149,0.048358,0.025672,0.128955,0.281791,-0.048780,-0.073726,-0.551476,-4.350906
